In [30]:
import networkx as nx
import matplotlib.pyplot as plt
import math
import itertools
from tqdm import tqdm_notebook as tqdm

In [2]:
# smaller graph for exact algorithms
_locations = \
      [(4, 4), # depot
       (2, 0), (8, 0), # locations to visit
       (3, 3), (6, 3),
       (5, 5),
       (1, 6),
       (0, 8), (7, 8)]

demands = [0, # depot
         1, 1, # row 0
         8, 8,
         1,
         1,
         8, 8]

capacities = [9, 9, 9, 9]

time_windows = \
        [(0, 0),
         (75, 85), (75, 85), # 1, 2
         (0, 10), (10, 20), # 7, 8
         (0, 10), # 9, 10
         (85, 95),
         (45, 55), (30, 40)] # 15, 16

In [3]:
G = nx.Graph(instance="Google OR example")

for i in range(len(_locations)):
    G.add_node(i, location=_locations[i], demand=demands[i], time_window=time_windows[i], status=0)

for i in range(len(_locations)):
    for j in range(len(_locations)):
        if i >= j:
            continue
        G.add_edge(i, j,
                   weight=math.sqrt(
                       (_locations[i][0]-_locations[j][0])**2
                       + (_locations[i][1]-_locations[j][1])**2))

In [95]:
def weight(node1: [float,float], node2: [float,float]):
    return math.sqrt((node1[0]-node2[0])**2
              + (node1[1]-node2[1])**2)

def powerset(iterable, lb=1):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(lb, len(s)+1))

SyntaxError: invalid syntax (<ipython-input-95-4d989e97a2d1>, line 1)

In [53]:
viable_routes = []
total_capacity = sum(capacities)
total_demand = sum(demands)
lower_bound = total_demand % max(capacities) or max(capacities)

for route in powerset(range(1, len(G.nodes))):
    demand = 0
    viable = True
    for i in route:
        if not viable:
            continue
        demand += demands[i]
        if demand > max(capacities):
            viable = False
            break
    else:
        if demand >= lower_bound:
            viable_routes += [[route, demand]]
    

In [96]:
G.get_edge_data(0,1)

{'weight': 4.47213595499958}

In [108]:
best_route = [[], 99999999999]
limit = 1000
for solution in tqdm(powerset(viable_routes)):
    attended_demand = sum([x[1] for x in solution])
    coverage = set(i for x in solution for i in x[0])
    # limit -= 1
    # if not limit:
    #     break
    # print (coverage)
    if attended_demand == total_demand <= total_capacity and len(coverage) == len(G.nodes) - 1:
        total_weight = sum([G.get_edge_data(x[0][i], x[0][i+1])["weight"] for x in solution for i in range(len(x[0])-1)])
        if total_weight < best_route[1]:
            best_route = [solution, total_weight]
best_route

[([(1, 3), 9], [(2, 4), 9], [(5, 8), 9], [(6, 7), 9]), 12.609448188596147]

In [74]:
x = [(1,2,3), (2,4), (4,5,6)]
set(i for sub in x for i in sub)

{1, 2, 3, 4, 5, 6}

In [102]:
total_weight = sum([G.get_edge_data(x[0][i], x[0][i+1])["weight"] for x in solution for i in range(len(x[0])-1)])
total_weight

87.153064201754

In [106]:
best_route


([(1, 8), 9], [(2, 7), 9], [(3, 6), 9], [(4, 5), 9])